<a href="https://colab.research.google.com/github/ritesh1776/Hand-Gesture-Recognition-using-GAN/blob/main/Gesture_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 70
LEARNING_RATE = 0.0002
NUM_CLASSES = 36
TRAIN_PATH = 'Train'
TEST_PATH = 'Test'

In [ ]:
import zipfile
import os

zip_path = '/content/Test.zip'  # Replace with the correct file path
extract_path = '/content/'  # Destination folder for extracted data

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the extracted contents
print("Extracted files:", os.listdir(extract_path))


Extracted files: ['.config', 'Test.zip', 'Train.zip', 'Test', 'sample_data']


In [ ]:
import zipfile
import os

zip_path = '/content/Train.zip'  # Replace with the correct file path
extract_path = '/content/'  # Destination folder for extracted data

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the extracted contents
print("Extracted files:", os.listdir(extract_path))


Extracted files: ['.config', 'Test.zip', 'Train.zip', 'Test', 'Train', 'sample_data']


In [ ]:
def load_data_by_class(path, img_size=IMG_SIZE):
    X, y = [], []
    classes = sorted(os.listdir(path))
    for label, class_folder in enumerate(classes):
        class_path = os.path.join(path, class_folder)
        for img_file in os.listdir(class_path):
            img = Image.open(os.path.join(class_path, img_file)).convert('L')
            img = img.resize(img_size)
            img = np.array(img)
            X.append(img)
            y.append(label)
    X = np.array(X).reshape(-1, img_size[0], img_size[1], 1)
    y = np.array(y)
    return X, y


X_train, y_train = load_data_by_class(TRAIN_PATH)
X_test, y_test = load_data_by_class(TEST_PATH)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0



In [ ]:
def build_generator(img_size=IMG_SIZE, num_classes=NUM_CLASSES):
    noise_input = layers.Input(shape=(100,))
    label_input = layers.Input(shape=(1,), dtype='int32')

    label_embedding = layers.Embedding(num_classes, 100)(label_input)
    label_embedding = layers.Flatten()(label_embedding)

    model_input = layers.multiply([noise_input, label_embedding])

    model = models.Sequential()
    x = layers.Dense(128 * 16 * 16, activation='relu')(model_input)
    x = layers.Reshape((16, 16, 128))(x)
    x = layers.UpSampling2D()(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.Activation('relu')(x)

    x = layers.UpSampling2D()(x)
    x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.Activation('relu')(x)

    img_output = layers.Conv2D(1, kernel_size=3, padding='same', activation='tanh')(x)

    generator = models.Model([noise_input, label_input], img_output)
    generator.summary()
    return generator

generator = build_generator()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 100)         │          3,600 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 100)            │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 100)            │              0 │ input_layer[0][0],     │
│                           │                        │                │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 32768)          │      3,309,568 │ multiply[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 16, 16, 128)    │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 32, 32, 128)    │              0 │ reshape[0][0]          │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 32, 32, 128)    │        147,584 │ up_sampling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 32, 32, 128)    │            512 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 32, 32, 128)    │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_1           │ (None, 64, 64, 128)    │              0 │ activation[0][0]       │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         73,792 │ up_sampling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 64, 64, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 64, 64, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 64, 64, 1)      │            577 │ activation_1[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,535,889 (13.49 MB)

 Trainable params: 3,535,505 (13.49 MB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
def build_discriminator(img_size=IMG_SIZE, num_classes=NUM_CLASSES):
    img_input = layers.Input(shape=(img_size[0], img_size[1], 1))
    label_input = layers.Input(shape=(1,), dtype='int32')

    label_embedding = layers.Embedding(num_classes, np.prod(img_size))(label_input)
    label_embedding = layers.Flatten()(label_embedding)
    label_embedding = layers.Reshape((img_size[0], img_size[1], 1))(label_embedding)

    x = layers.Concatenate()([img_input, label_embedding])

    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Flatten()(x)
    validity_output = layers.Dense(1, activation='sigmoid')(x)

    discriminator = models.Model([img_input, label_input], validity_output)
    discriminator.summary()
    return discriminator

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(LEARNING_RATE), metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 4096)        │        147,456 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 4096)           │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 64, 64, 1)      │              0 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 64, 2)      │              0 │ input_layer_2[0][0],   │
│                           │                        │                │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 32, 32, 64)     │          1,216 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 32, 32, 64)     │              0 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 32, 32, 64)     │              0 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 16, 16, 128)    │         73,856 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 16, 16, 128)    │              0 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 16, 16, 128)    │              0 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 32768)          │              0 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │         32,769 │ flatten_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 255,297 (997.25 KB)

 Trainable params: 255,297 (997.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def build_gan(generator, discriminator):
    noise_input = layers.Input(shape=(100,))
    label_input = layers.Input(shape=(1,), dtype='int32')

    img_output = generator([noise_input, label_input])
    validity_output = discriminator([img_output, label_input])

    gan = models.Model([noise_input, label_input], validity_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(LEARNING_RATE))

    return gan

gan = build_gan(generator, discriminator)


In [ ]:
def train_gan(generator, discriminator, gan, X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs, labels = X_train[idx], y_train[idx]
        real_labels = np.ones((half_batch, 1))
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_labels = np.random.randint(0, NUM_CLASSES, half_batch)
        fake_imgs = generator.predict([noise, gen_labels])
        fake_labels = np.zeros((half_batch, 1))
        d_loss_real = discriminator.train_on_batch([real_imgs, labels], real_labels)
        d_loss_fake = discriminator.train_on_batch([fake_imgs, gen_labels], fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.ones((batch_size, 1))
        sampled_labels = np.random.randint(0, NUM_CLASSES, batch_size)
        g_loss = gan.train_on_batch([noise, sampled_labels], valid_y)
        print(f"{epoch+1}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

train_gan(generator, discriminator, gan, X_train, y_train)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/70, D Loss: 0.7407864332199097, G Loss: [array(0.72520083, dtype=float32), array(0.72520083, dtype=float32), array(0.15625, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


2/70, D Loss: 0.6899281740188599, G Loss: [array(0.6914458, dtype=float32), array(0.6914458, dtype=float32), array(0.328125, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
3/70, D Loss: 0.6559301614761353, G Loss: [array(0.66049725, dtype=float32), array(0.66049725, dtype=float32), array(0.38541666, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/70, D Loss: 0.6375690698623657, G Loss: [array(0.64255476, dtype=float32), array(0.64255476, dtype=float32), array(0.4140625, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5/70, D Loss: 0.6085702180862427, G Loss: [array(0.61439914, dtype=float32), array(0.61439914, dtype=float32), array(0.43125, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
6/70, D Loss: 0.5868613719940186, G Loss: [array(0.5929244, dtype=float32), array(0.5929244, dtype=float32), array(0.44270834, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/70, D Loss: 0.5657067894935608, G Loss: [array(0.5718842, dtype=float32), array(0.5718842, dt

In [ ]:
def augment_data_with_gan(generator, class_label, num_samples):
    noise = np.random.normal(0, 1, (num_samples, 100))
    labels = np.full((num_samples,), class_label)
    generated_imgs = generator.predict([noise, labels])
    return generated_imgs


In [ ]:
augmented_imgs_class_0 = augment_data_with_gan(generator, class_label=0, num_samples=100)
augmented_imgs_class_1= augment_data_with_gan(generator, class_label=1, num_samples=100)
augmented_imgs_class_2 = augment_data_with_gan(generator, class_label=2, num_samples=100)
augmented_imgs_class_3 = augment_data_with_gan(generator, class_label=3, num_samples=100)
augmented_imgs_class_4 = augment_data_with_gan(generator, class_label=4, num_samples=100)
augmented_imgs_class_5 = augment_data_with_gan(generator, class_label=5, num_samples=100)
augmented_imgs_class_6 = augment_data_with_gan(generator, class_label=6, num_samples=100)
augmented_imgs_class_7 = augment_data_with_gan(generator, class_label=7, num_samples=100)



4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [ ]:
def build_cnn(img_size=IMG_SIZE, num_classes=NUM_CLASSES):
    model = models.Sequential()

    model.add(layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(img_size[0], img_size[1], 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(LEARNING_RATE), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

cnn = build_cnn()
cnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0822 - loss: 3.4804 - val_accuracy: 0.2604 - val_loss: 2.6589
Epoch 2/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3563 - loss: 2.2144 - val_accuracy: 0.5688 - val_loss: 1.4626
Epoch 3/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6039 - loss: 1.2736 - val_accuracy: 0.6812 - val_loss: 1.0147
Epoch 4/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7670 - loss: 0.7588 - val_accuracy: 0.8181 - val_loss: 0.5987
Epoch 5/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8568 - loss: 0.4815 - val_accuracy: 0.8833 - val_loss: 0.4035
Epoch 6/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8974 - loss: 0.3409 - val_accuracy: 0.9167 - val_loss: 0.2955
Epoch 7/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9333 - loss: 0.2329 - val_accuracy: 0.9368 - val_loss: 0.2255
Epoch 8/70
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9674 - loss: 0.1428 - val_accuracy: 0.

In [ ]:
# Write code to test the model on the test set i.e X_test and y_test
# and print the accuracy of the model on the test set
test_loss, test_acc = cnn.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8155 - loss: 1.6332
Test Accuracy: 0.8633333444595337


In [ ]:
cnn.save('cnn_model_Gan.h5')